In [1]:
"""
  We use the upper Hessenberg Frank matrix
        A[i,j] =     0     for j < i-1, 
                 n+1-i     for j = i-1 
                 n+1-j     for j \geq i              

  to illustrate the role of the condition number and its relation with the
  loss of accuracy when solving linear systems 
  (due to rounding errors, independently of the method used!)

  Remark: det(A)=1 but it is ill-conditioned!

  Explanation: Backward error analysis formula 
               (we assume Delta_A =0, and that there is only representation error in b)
"""

import numpy as np

def frank(n):
  A = np.zeros((n,n))

  for i in range (0,n):
    for j in range (0,n):
      if(i<=j):
        A[i,j] = float(n-j)
      elif(j==i-1):
        A[i,j] = float(n-j-1)

  return A



"""
  MAIN
"""

n=5;
f=frank(n)
print(f)
print(np.linalg.det(f))


print('\n Condition number, upper Hessenberg FRANK matrix\n')

nmin,nmax=map(int,input(' nmin,nmax?\n').split());

eps=1.e-18  #we assume that the error in b is Delta_b = eps_machine (double precision, internal precision)

outfile = open("frank.out","w")

print('n \t cond(A) \t\t eps*cond(A) \t\t xsol-x \n'); 
for n in range (nmin,nmax+1):
   A=frank(n)

   x=np.ones(n)
   b=np.dot(A,x)
   xsol=np.linalg.solve(A,b) ### We ignore the error of the gaussian elimination
   print('%2d %24.16e %24.16e %24.16e' % (n,np.linalg.cond(A),eps*np.linalg.cond(A), np.linalg.norm(xsol-x)/np.linalg.norm(x)))
   outfile.write('%2d %24.16e %24.16e %24.16e\n' % (n,np.linalg.cond(A),eps*np.linalg.cond(A), np.linalg.norm(xsol-x)/np.linalg.norm(x)))

outfile.close()

[[5. 4. 3. 2. 1.]
 [4. 4. 3. 2. 1.]
 [0. 3. 3. 2. 1.]
 [0. 0. 2. 2. 1.]
 [0. 0. 0. 1. 1.]]
1.000000000000003

 Condition number, upper Hessenberg FRANK matrix

 nmin,nmax?
2,4


ValueError: invalid literal for int() with base 10: '2,4'

In [2]:
import numpy as np

outfile = open("hilbert.out","w")

for n in range (2,30):
    A=np.zeros((n,n))
    for i in range(0,n):
        for j in range (0,n):
            A[i,j]=1./(i+j+1)

    kA=np.linalg.norm(A,2)*np.linalg.norm(np.linalg.inv(A),2)
    kkA=np.linalg.cond(A,2)
    print('%2d %24.16e %24.16e\n' %(n,kA,kkA))
    outfile.write('%2d %24.16e %24.16e\n' % (n,kA,kkA))

outfile.close()

 2   1.9281470067903975e+01   1.9281470067903971e+01

 3   5.2405677758606271e+02   5.2405677758606441e+02

 4   1.5513738738933602e+04   1.5513738738929240e+04

 5   4.7660725024576450e+05   4.7660725024259434e+05

 6   1.4951058642074950e+07   1.4951058642254665e+07

 7   4.7536735646774173e+08   4.7536735658312899e+08

 8   1.5257576321957926e+10   1.5257575538060041e+10

 9   4.9315378601241656e+11   4.9315375644687622e+11

10   1.6026019477413041e+13   1.6024416992541715e+13

11   5.2342065664873800e+14   5.2226779392803350e+14

12   1.7715806936210972e+16   1.7514731907091464e+16

13   3.3028536593780403e+17   3.3441434973384612e+18

14   4.3544723094505190e+17   6.2007862631614438e+17

15   4.3803470904383226e+17   3.6743929534679738e+17

16   3.1464399162753464e+18   7.8654677784316454e+17

17   9.5904030774353024e+17   1.2636843426660521e+18

18   7.0406149242160448e+17   2.2446309929189128e+18

19   7.4602581749320192e+17   6.4719539765415905e+18

20   2.0859944216801879e+18 

In [3]:
"""
  We compare the efectivenes of solving Ax=b using
  1) x=A^{-1} b
  2) GEPP (Gaussian ellimination with partial pivoting)
"""

import numpy as np

n=25     #dimension
nsys=50  #number of systems

nor1=np.zeros(nsys) #norm-wise relative error for each system using A^-1 to solve it
nor2=np.zeros(nsys) #norm-wise relative error for each system using GEPP to solve it

for isys in range (0,nsys):
    A=np.zeros((n,n))
    Q=np.zeros((n,n))
    x=np.random.normal(0,1,n)
    
    for i in range (0,n):
        for j in range (0,n):
            A[i,j]=np.random.normal(0,1)
    
    Q,R=np.linalg.qr(A)
    D=np.eye(n)         #Returns a 2d array with ones in the diagonal and zeros outside
    D[0,0]=9e7          #Value of the condition number 
    A=np.dot(np.dot(Q.T,D),Q)
    b=np.dot(A,x)
   
    
    iA=np.linalg.inv(A); x1=np.dot(iA,b)   #Direct inversion method
    x2=np.linalg.solve(A,b)                #GEPP (computes LU factorization and solves the linear system)
    
    r1=np.dot(A,x1)-b;  r2=np.dot(A,x2)-b  #residues
    nr1=np.linalg.norm(r1,np.inf);  nr2=np.linalg.norm(r2,np.inf)  #residue norms
    nA=np.linalg.norm(A,np.inf)  #norm of A
    nb=np.linalg.norm(b,np.inf)  #norm of b
    nx1=np.linalg.norm(x1,np.inf);  nx2=np.linalg.norm(x2,np.inf)  #solution norms
    nor1[isys]=nr1/(nA*nx1+nb);  nor2[isys]=nr2/(nA*nx2+nb)  #array of normwise backward errors

print(nor1)
print(nor2)
print('\n Direct invert method solutions, min and max normwise backward error: \n %24.16e %24.16e \n\n GEPP (LU), min and max normwise backward error: \n %24.16e %24.16e\n' % (np.amin(nor1), np.amax(nor1), np.amin(nor2), np.amax(nor2)))

[4.60866498e-11 2.01361728e-11 2.14757560e-10 3.25169569e-11
 4.85564162e-10 8.86339022e-11 1.15987631e-10 5.72879790e-12
 1.02969977e-11 2.37963914e-10 1.41138121e-10 3.69860670e-11
 1.48055384e-12 3.02987839e-11 1.99868180e-10 9.69281392e-11
 4.21828235e-12 4.35849950e-10 7.64353354e-11 8.16790259e-11
 4.35089615e-10 3.91973658e-10 2.20050832e-10 7.35515376e-11
 2.30575062e-11 3.30031736e-10 6.93422944e-12 9.03239437e-10
 9.11767870e-12 1.03460647e-09 5.39238196e-10 1.96234066e-10
 2.52961292e-10 5.47148536e-10 1.31546118e-11 2.31536362e-11
 2.11687471e-10 3.86702352e-10 8.88234583e-11 3.78821529e-11
 6.35998435e-10 1.38302351e-09 7.85872279e-11 1.63852871e-11
 5.14096018e-10 2.91215505e-10 5.82591597e-10 7.59383858e-11
 1.91921878e-10 3.20356995e-11]
[8.75225773e-18 2.03627559e-17 8.37468411e-18 2.05475350e-17
 3.66923114e-17 1.95324457e-17 2.07129203e-17 2.62494491e-17
 1.27769321e-17 1.52393501e-17 2.04976046e-17 8.65799430e-18
 7.48980069e-18 2.71616901e-17 2.28323785e-17 1.47878

In [4]:
"""
  We compare the efectivenes of solving Ax=b using
  1) x=A^{-1} b
  2) GEPP (Gaussian ellimination with partial pivoting)
"""

import numpy as np
import time

ini=time.perf_counter()

nmax=100
nsys=10000

outfile=open("inv_time.out","w");
    
outfile.write('# n = dimension\n') 
outfile.write('# t1 = direct inversion method time (in sec) for solving %d systems of dimension n\n' % (nsys)) 
outfile.write('# t2 = Gauss (LU) time (in sec) for solving %d systems of dimension n\n' % (nsys)) 
outfile.write('# file: n t1 t2\n')

for n in range (2,nmax):
    print("# starting n=%d" % (n))
    Alist=np.zeros((nsys,n,n))
    for isys in range (0,nsys):
        A=np.zeros((n,n))
        Q=np.zeros((n,n))
        x=np.random.normal(0,1,n)
        
        for i in range (0,n):
            for j in range (0,n):
                A[i,j]=np.random.normal(0,1)
        
        Q,R=np.linalg.qr(A)
        B=np.eye(n)         #Returns a 2d array with ones in the diagonal and zeros outside
        B[0,0]=9e7          #Value of the condition number 
        Alist[isys]=np.dot(np.dot(Q.T,B),Q)
        b=np.dot(Alist[isys],x)
        
    print("# %d systems of dimension %d with k(A)=%e have been randomly generated" % (nsys,n,B[0,0]))
    start1 = time.perf_counter()
    for isys in range (0,nsys):
        iA=np.linalg.inv(Alist[isys]); x1=np.dot(iA,b)   #Direct inversion method
    end1 = time.perf_counter()
    
    start2 = time.perf_counter()
    for isys in range (0,nsys):
        x2=np.linalg.solve(Alist[isys],b)                #GEPP (computes LU factorization and solves the linear system)
    end2 = time.perf_counter()
    print('%d %24.16e %24.16e \n ' % (n,end1 -start1,end2-start2))
    outfile.write('%d %24.16e %24.16e\n' % (n,end1 -start1,end2-start2))
        

fin = time.perf_counter()
outfile.write('#Total time (sec): %24.16e\n' % (fin -ini))
outfile.close()

# starting n=2
# 10000 systems of dimension 2 with k(A)=9.000000e+07 have been randomly generated
2   1.7471560900000327e-01   1.2042008200000964e-01 
 
# starting n=3
# 10000 systems of dimension 3 with k(A)=9.000000e+07 have been randomly generated
3   1.8010480500001336e-01   1.0502370399998995e-01 
 
# starting n=4
# 10000 systems of dimension 4 with k(A)=9.000000e+07 have been randomly generated
4   2.1172481499999662e-01   1.0858569900000248e-01 
 
# starting n=5
# 10000 systems of dimension 5 with k(A)=9.000000e+07 have been randomly generated
5   2.1306063599999447e-01   1.3989321700000801e-01 
 
# starting n=6
# 10000 systems of dimension 6 with k(A)=9.000000e+07 have been randomly generated
6   2.2230403699998647e-01   1.4478632699999139e-01 
 
# starting n=7
# 10000 systems of dimension 7 with k(A)=9.000000e+07 have been randomly generated
7   2.3993757100001289e-01   1.3933346799998958e-01 
 
# starting n=8
# 10000 systems of dimension 8 with k(A)=9.000000e+07 have been ran

# 10000 systems of dimension 55 with k(A)=9.000000e+07 have been randomly generated
55   9.9167688500006079e-01   5.3059729599999628e-01 
 
# starting n=56
# 10000 systems of dimension 56 with k(A)=9.000000e+07 have been randomly generated
56   9.8553042999992613e-01   5.4797983499997827e-01 
 
# starting n=57
# 10000 systems of dimension 57 with k(A)=9.000000e+07 have been randomly generated
57   1.2333306570001241e+00   9.2577499300000454e-01 
 
# starting n=58
# 10000 systems of dimension 58 with k(A)=9.000000e+07 have been randomly generated
58   1.5189711199998328e+00   7.3959559499985517e-01 
 
# starting n=59
# 10000 systems of dimension 59 with k(A)=9.000000e+07 have been randomly generated
59   1.2752329519998966e+00   6.5480881700000282e-01 
 
# starting n=60
# 10000 systems of dimension 60 with k(A)=9.000000e+07 have been randomly generated
60   1.2658921359998203e+00   6.1514139099995191e-01 
 
# starting n=61
# 10000 systems of dimension 61 with k(A)=9.000000e+07 have been

KeyboardInterrupt: 

In [ ]:
import numpy as np

outfile=open("vander.out","w")

for n in range (2,51):
    alph=np.zeros(n)
    for j in range (1,n+1):
        alph[j-1] = 1-2.*(j-1)/(n-1)
    
    V=np.zeros((n,n))
    for i in range (0,n):
        for j in range (0,n):
            V[i,j] = alph[j]**i

    print(V)
    kV=np.linalg.norm(V,2)*np.linalg.norm(np.linalg.inv(V),2)
    print('%2d %24.16e\n'%(n,kV))
    outfile.write('%2d %24.16e %24.16e\n' %(n,np.linalg.norm(V,2),kV))

outfile.close()